##### Copyright 2023 HrFlow's AI Research Department

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2023 HrFlow's AI Research Department. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<p>
<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/Riminder/python-hrflow-api/blob/feature/add-upload-ratings-colab/examples/colab/upload_tracking_data.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/Riminder/python-hrflow-api/blob/feature/add-upload-ratings-colab/examples/colab/upload_tracking_data.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td><td>
  <a target="_blank"  href="https://www.hrflow.ai/book-us">
    <img width=32px src="https://gblobscdn.gitbook.com/spaces%2F-M1L6Hspq8r9LXd5_gIC%2Favatar-1586188377926.png?generation=1586188378327930&alt=media" />Get an account</a>
</td></table>
<br>
</p>

Welcome to this Google Colaboratory tutorial for developers. **In only 3 steps**, we'll help you **upload your profile, job, and tracking data to HrFlow.ai**. This will enable you to test the powerful **HrFlow.ai Retraining feature**.

Before we proceed, please ensure that you have created a source and a board in HrFlow.ai to store your data. You can find detailed instructions on how to create them through the following links:
- **Create your source**: Connectors Source Documentation
- **Create your board**: Connectors Board Documentation

Now, let's take a quick look at how this notebook is organized:
1. **👷 Uploading Profile Data**: Build and upload candidate profile information.
2. **🛠 Uploading Job Data**: Create and upload job details.
3. **🛤 Uploading Tracking Data**: Monitor candidate progress by uploading tracking data.

Let's get started and harness the capabilities of HrFlow.ai!

# Getting Started

In [1]:
!pip install --quiet tqdm
!pip install --quiet hrflow

In [2]:
from hrflow import Hrflow
from tqdm import tqdm


SOURCE_KEY = "YOUR_SOURCE_KEY"
BOARD_KEY = "YOUR_BOARD_KEY"

client = Hrflow(
    api_secret="YOUR_API_SECRET",
    api_user="YOUR_API_USER"
)

# 1. 👷 Uploading Profile Data


**Please use the code 'profiles = load_profiles()' in this cell to load all your profiles.**

To provide you with a better understanding of how this notebook works, we have included a Dummy Profile as an example in the second cell. This will help you familiarize yourself with the structure and format of the data.

In [3]:
def load_profiles():
  """
  Loads and formats your profile data into an HrFlow.ai Profile object.

  Note: You need to implement the mapping between your data and the HrFlow.ai Profile object.

  Please refer to the HrFlow.ai Profile object documentation for detailed information:
  https://developers.hrflow.ai/reference/the-profile-object

  Returns:
      List[Dict]: The formatted Profile object.

  Raises:
      NotImplementedError: This function is not implemented yet.
  """
  raise NotImplementedError("You need to implement this function and do the mapping between your data and the HrFlow.ai Profile object.")

In [4]:
profiles = [
  {
    "reference": "DUMMY_PROFILE",
    "info": {
      "full_name": "DUMMY PROFILE",
      "first_name": "DUMMY",
      "last_name": "PROFILE",
      "email": "dummy_profile@hrflow.ai",
      "phone": "+16506815000",
      "location": {
        "text": "Los Angeles, USA",
        "lat": 34.05358,
        "lng": -118.24546,
      },
      "gender": "male",
      "summary": 'Forged by HrFlow.ai...',
    },
    "text": "DUMMY PROFILE\nForged by HrFlow.ai...",
    "experiences": [],
    "educations": [],
    "skills": [],
    "languages": [],
    "certifications": [],
    "courses": [],
    "tasks": [],
    "interests": [],
  }
]

for profile in tqdm(profiles):
    response = client.profile.storing.add_json(SOURCE_KEY, profile)
print(response)

100%|██████████| 1/1 [00:01<00:00,  1.34s/it]

{'code': 201, 'message': 'Profile created', 'data': {'source_key': 'fe74630314b32622cba0027dcbcf4b4db512e477', 'id': 14996699, 'key': 'b32b1b501d0fb6be701b2685caf38d99f950df57', 'reference': 'DUMMY_PROFILE', 'consent_algorithmic': {'owner': {'parsing': False, 'revealing': False, 'embedding': False, 'searching': False, 'scoring': False, 'reasoning': False}, 'controller': {'parsing': False, 'revealing': False, 'embedding': False, 'searching': False, 'scoring': False, 'reasoning': False}}, 'source': {'key': 'fe74630314b32622cba0027dcbcf4b4db512e477', 'name': 'test-upload-notebook', 'type': 'api', 'subtype': 'http_api', 'environment': 'production'}, 'archive': None, 'archived_at': None, 'updated_at': '2023-06-22T01:33:50+0000', 'created_at': '2023-06-22T01:33:50+0000', 'info': {'full_name': 'DUMMY PROFILE', 'first_name': 'DUMMY', 'last_name': 'PROFILE', 'email': 'dummy_profile@hrflow.ai', 'phone': '+16506815000', 'date_birth': None, 'location': {'text': 'Los Angeles, USA', 'lat': 34.05358,

# 2. 🛠 Uploading Job Data

**Please use the code 'jobs = load_jobs()' in the following cell to load all your jobs.**

To provide you with a better understanding of how this notebook works, we have included a Dummy JOB as an example in the second cell. This will help you familiarize yourself with the structure and format of the data.

In [5]:
def load_jobs():
  """
  Loads and formats your job data into an HrFlow.ai Job object.

  Note: You need to implement the mapping between your data and the HrFlow.ai Job object.

  Please refer to the HrFlow.ai Job object documentation for detailed information:
  https://developers.hrflow.ai/reference/the-job-object

  Returns:
      List[Dict]: The formatted Job object.

  Raises:
      NotImplementedError: This function is not implemented yet.
  """
  raise NotImplementedError("You need to implement this function and do the mapping between your data and the HrFlow.ai Job object.")

In [6]:
jobs = [
  {
    "reference": "DUMMY_JOB",
    "name": "DUMMY_JOB",
    "summary": "Forged by HrFlow.ai",
    "location": {
      "text": "Paris (75)",
      "geopoint": {
        "lat": 48.86131,
        "lon": 2.34054
      },
    },
    "culture": "",
    "responsibilities": "",
    "requirements": "",
    "benefits": "",
    "interviews": "",
    "sections": [],
    "skills": [],
    "languages": [],
  }
]

for job in tqdm(jobs):
  response = client.job.storing.add_json(BOARD_KEY, job)
print(response)


100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

{'code': 201, 'message': 'Job created', 'data': {'id': 1278943, 'key': '9bc72a570845677f4f2e860eb95f78c1d7d352bb', 'reference': 'DUMMY_JOB', 'board_key': 'f040de20e7d66f748e1d2141fdf950d53478bc6e', 'board': {'key': 'f040de20e7d66f748e1d2141fdf950d53478bc6e', 'name': 'test-upload-notebook', 'type': 'api', 'subtype': 'http_api', 'environment': 'production'}, 'name': 'DUMMY_JOB', 'url': None, 'picture': None, 'summary': 'Forged by HrFlow.ai', 'location': {'text': 'Paris (75)', 'lat': None, 'lng': None, 'gmaps': None, 'fields': None}, 'archive': None, 'archived_at': None, 'updated_at': '2023-06-22T01:34:10+0000', 'created_at': '2023-06-22T01:34:10+0000', 'sections': [], 'culture': '', 'responsibilities': '', 'requirements': '', 'benefits': '', 'interviews': '', 'skills': [], 'languages': [], 'certifications': None, 'courses': None, 'tasks': None, 'tags': None, 'metadatas': None, 'ranges_float': None, 'ranges_date': None}}


# 3. 🛤 Uploading Tracking Data

Please use the code 'trackings = load_trackings()' in this cell to load all your trackings.

To provide you with a better understanding of how this notebook operates, we have included a dummy tracking example. This will help you grasp the structure and format of the data.

Please note that the 'profile_reference' and 'job_reference' fields are being used to upload your profiles and jobs, respectively. This information is crucial for constructing the tracking objects accurately.

In [7]:
def load_trackings():
    """
    Loads and formats your tracking data into an HrFlow.ai Tracking object.

    Note: You need to implement the mapping between your data and the HrFlow.ai Tracking object.

    The schema for the HrFlow.ai Tracking object is as follows:
    Tracking = {
        "action": ["view", "apply", "hire"],
        "role": ["recruiter", "candidate", "employee", "manager"]
        "source_key": str
        "profile_reference": str,
        "board_key": str
        "job_reference": str
    }

    Returns:
        List[Tracking]: The formatted Tracking object.

    Raises:
        NotImplementedError: This function is not implemented yet.
    """
    raise NotImplementedError("You need to implement this function and map your data to the HrFlow.ai Tracking object.")

In [8]:
trackings = [
  {
    "action": "apply",
    "role": "candidate",
    "source_key": SOURCE_KEY,
    "profile_reference": "DUMMY_PROFILE",
    "board_key": BOARD_KEY,
    "job_reference": "DUMMY_JOB"
  }
]

for tracking in tqdm(trackings):
  response = client.tracking.post(**tracking)
print(response)

100%|██████████| 1/1 [00:01<00:00,  1.18s/it]

{'code': 201, 'message': 'Tracking created', 'data': {'action': 'apply', 'duration': None, 'team_key': 'fc9d40fd60e679119130ea74ae1d34a3e22174f2', 'source_key': 'fe74630314b32622cba0027dcbcf4b4db512e477', 'profile_key': 'b32b1b501d0fb6be701b2685caf38d99f950df57', 'profile_reference': 'DUMMY_PROFILE', 'board_key': 'f040de20e7d66f748e1d2141fdf950d53478bc6e', 'job_key': '9bc72a570845677f4f2e860eb95f78c1d7d352bb', 'job_reference': 'DUMMY_JOB', 'author_email': None, 'role': 'candidate', 'comment': None, 'updated_at': '2023-06-22T01:34:18+0000', 'created_at': '2023-06-22T01:34:18+0000'}}
